In [30]:
import tensorflow as tf
import os
import sys


from preprocess import create_data_generators, get_class_map

print("TensorFlow version:", tf.__version__)




TensorFlow version: 2.20.0


In [31]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10

In [32]:

BASE_DIR = "../dataset" 
TRAIN_DIR = os.path.join(BASE_DIR, "train")
TEST_DIR = os.path.join(BASE_DIR, "test")

print(f"Train path exist: {os.path.exists(TRAIN_DIR)}")
print(f"Test path exist: {os.path.exists(TEST_DIR)}")

Train path exist: True
Test path exist: True


In [33]:
train_generator, test_generator = create_data_generators(
    TRAIN_DIR,
    TEST_DIR,
    IMG_SIZE,
    BATCH_SIZE
    )

labels = get_class_map(train_generator)

Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [34]:

X_batch, y_batch = next(train_generator) 
print("Batch image shape:", X_batch.shape)  

print("\nLoading Test Data:")

X_batch, y_batch = next(test_generator)  
print("Batch image shape:", X_batch.shape) 


Batch image shape: (32, 224, 224, 3)

Loading Test Data:
Batch image shape: (32, 224, 224, 3)


In [35]:
import tensorflow as tf

num_classes = len(labels)

model_baseline = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model_baseline.summary()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,476 (42.61 MB)

 Trainable params: 11,169,476 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
model_baseline.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [37]:
import time
import math

steps_per_epoch = math.ceil(train_generator.samples / BATCH_SIZE)
validation_steps = math.ceil(test_generator.samples / BATCH_SIZE)

start = time.time()
history_baseline = model_baseline.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_generator,
    validation_steps=validation_steps,
    epochs=EPOCHS
)
train_time_sec = time.time() - start

print("Training time (sec):", round(train_time_sec, 2))

Epoch 1/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 118s 1s/step - accuracy: 0.4387 - loss: 1.2763 - val_accuracy: 0.2183 - val_loss: 1.7477
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - accuracy: 0.5251 - loss: 1.0670 - val_accuracy: 0.2919 - val_loss: 1.8672
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - accuracy: 0.5659 - loss: 0.9875 - val_accuracy: 0.2843 - val_loss: 2.6251
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 93s 1s/step - accuracy: 0.6167 - loss: 0.8876 - val_accuracy: 0.3426 - val_loss: 2.1395
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 93s 1s/step - accuracy: 0.6192 - loss: 0.8674 - val_accuracy: 0.3426 - val_loss: 2.6424
Epoch 6/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 92s 1s/step - accuracy: 0.6533 - loss: 0.8034 - val_accuracy: 0.3629 - val_loss: 2.8743
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.6662 - loss: 0.7679 - val_accuracy: 0.3731 - val_loss: 3.2196
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.6735 - loss: 0.7399 - val_accuracy: 0.4112 - val_lo

In [38]:
baseline_loss, baseline_acc = model_baseline.evaluate(
    test_generator,
    steps=validation_steps
)
print("Baseline Loss:", baseline_loss)
print("Baseline Accuracy:", baseline_acc)

13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 271ms/step - accuracy: 0.3655 - loss: 3.6030
Baseline Loss: 3.6030385494232178
Baseline Accuracy: 0.36548224091529846


In [39]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# reset generator to start
test_generator.reset()

y_prob = model_baseline.predict(test_generator, steps=validation_steps)
y_pred = np.argmax(y_prob, axis=1)

y_true = test_generator.classes  # integer labels in directory order
class_names = list(test_generator.class_indices.keys())

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_names))

13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 277ms/step
Confusion Matrix:
 [[ 11   7  77   5]
 [  2   8 100   5]
 [  2   0 103   0]
 [  5   2  45  22]]

Classification Report:

                  precision    recall  f1-score   support

    glioma_tumor       0.55      0.11      0.18       100
meningioma_tumor       0.47      0.07      0.12       115
        no_tumor       0.32      0.98      0.48       105
 pituitary_tumor       0.69      0.30      0.42        74

        accuracy                           0.37       394
       macro avg       0.51      0.36      0.30       394
    weighted avg       0.49      0.37      0.29       394



In [40]:
import os
os.makedirs("../models", exist_ok=True)

model_baseline.save("../models/baseline_cnn.keras")
print("Saved to ../models/baseline_cnn.keras")

Saved to ../models/baseline_cnn.keras
